In [ ]:
# import sys
# !{sys.executable} -m pip install PyPDF2

import requests
from bs4 import BeautifulSoup
import time
import io
# import PyPDF2
import json
import concurrent.futures
import pandas as pd

In [ ]:
def new_car_by_category():

    url = "https://www.pakwheels.com/new-cars/"

    headers = {
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_source = response.text
    soup = BeautifulSoup(html_source, 'html.parser')

    alldetails = []
    extracted_links = []
    #Find all 'a' tags with the specified class for each make/company of cars
    ul = soup.find_all('ul',class_='make-list col-sm-2 list-unstyled new-car-list')

    for each in ul:
      #Extract href attribute from each link
      extracted_links.append(url + each.find('a').get('href').split('/')[2])

    for link in extracted_links:

      response = requests.get(link, headers=headers)
      html_source = response.text
      soup = BeautifulSoup(html_source, 'html.parser')

      #find all new car links of each make/company
      ul = soup.find_all('ul', class_='list-unstyled model-list row item clearfix')[0]
      li = ul.find_all('li')
      for each in li:
        #Extract href attribute from each link
        carurl = link + "/" + each.find('a').get('href').split('/')[3]

        response = requests.get(carurl, headers=headers)
        html_source = response.text
        soup = BeautifulSoup(html_source, 'html.parser')

        cardetails={}
        cardetails['title'] = carurl.split('/')[4] + " " + carurl.split('/')[5]

        #extract all data from table
        try:
          info_table = soup.find('table', class_='bike-version-detailscont')
          info_rows = info_table.find_all('tr')
          # Iterate through table rows
          for row in info_rows:
              columns = row.find_all('td')
              if len(columns) == 2:
                  label = columns[0].text.strip()
                  value = columns[1].text.strip()
                  cardetails[label] = value
        except:
          print('Table not found for ' + cardetails['title'])
          break

        print(cardetails)


        # litags = soup.find('ul',class_='gallery light-gallery list-unstyled cS-hidden').find_all('li')
        # images = []
        # for li in litags:
        #   images.append(li.get('data-src'))
        # cardetails['images'] = images
        alldetails.append(cardetails)

     #Create a DataFrame from the car details
    df = pd.DataFrame(alldetails)
    # Save the DataFrame to an Excel file
    df.to_excel('new_car_details.xlsx', index=False)

    print("Car details saved to 'new_car_details.xlsx'")

In [ ]:
new_car_by_category()